# IMD1114 - IA Generativa

### Professor: Dr. Leonardo Enzo Brito da Silva

### Aluno: João Antonio Costa Paiva Chagas

# Tarefa:

1. Implementar uma WCGAN condicional e treinar o modelo com o conjunto de dados Fashion MNIST.

    a. Mostre os gráficos das funções de perda do gerador e do discriminador.

    b. Mostre imagens geradas com o modelo.

2. Interpolar entre vetores de ruído e mostrar as imagens intermediárias considerando:

    a. z fixo e interpolacão linear entre classes [c1, c2].

    b. classe c fixa e interpolaćão linear entre [z1, z2].

    c. interpolaćão linear entre [z1, z2] e [c1, c2].

3. Fixe o vetor de ruído e altere apenas o rótulo para observar como a imagem muda.

**Entregáveis**:
1. Notebook `.ipynb`.
2. Relatório `.pdf`:

    - Reporte e comente os resultados no relatório.

    - Incluir imagens geradas.

### Importações:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import os

from google.colab import drive

### Ambiente:

In [ ]:
if not os.path.exists('files'):
    os.makedirs('files')

In [ ]:
drive.mount('/content/drive')

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

In [ ]:
FMNIST_LABELS = {
    0: "T-shirt/top", 1: "Trouser", 2: "Pullover", 3: "Dress", 4: "Coat",
    5: "Sandal", 6: "Shirt", 7: "Sneaker", 8: "Bag", 9: "Ankle boot"
}

In [ ]:
MNIST_LABELS = {
    0: "0", 1: "1", 2: "2", 3: "3", 4: "4",
    5: "5", 6: "6", 7: "7", 8: "8", 9: "9"
}

### Dados:

In [ ]:
def prepare_data(batch_size, dataset_choice='FMNIST'):
    """Carrega o dataset escolhido e retorna um DataLoader."""
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)) # Normaliza as imagens para o intervalo [-1, 1]
    ])

    if dataset_choice == 'FMNIST':
        training_data = datasets.FashionMNIST(
            root="data", train=True, download=True, transform=transform
        )
    elif dataset_choice == 'MNIST':
        training_data = datasets.MNIST(
            root="data", train=True, download=True, transform=transform
        )

    return DataLoader(
        training_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,
        pin_memory=True
    )


### Plots:

In [ ]:
def plot_loss_history(loss_g, loss_c):
    """Plota o histórico de perdas do Gerador e do Crítico."""
    plt.figure(figsize=(10, 5))
    plt.title("Perda do Gerador e Crítico Durante o Treinamento")
    plt.plot(loss_g, label="Gerador")
    plt.plot(loss_c, label="Crítico")
    plt.xlabel("Épocas")
    plt.ylabel("Perda (Wasserstein Distance Estimate)")
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def plot_image_grid(images, titles=None, n_cols=5, figsize=(12, 6)):
    """
    Plota uma grade de imagens com títulos opcionais.
    Esta função substitui plot_multiple_images, plotar_imagens e outras.
    """
    if not images:
        print("Nenhuma imagem para plotar.")
        return

    n_rows = (len(images) - 1) // n_cols + 1
    plt.figure(figsize=figsize)

    for index, img in enumerate(images):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(img.reshape(28, 28), cmap="gray")
        plt.axis("off")
        if titles and index < len(titles):
            plt.title(titles[index])

    plt.tight_layout()
    plt.show()

### Checkpoint

In [ ]:
def save_checkpoint(epoch, generator, critic, opt_g, opt_c, path="/content/drive/MyDrive/gan_checkpoint.pth"):
    """Salva o estado do treinamento."""
    print(f"==> Salvando checkpoint da época {epoch}...")
    checkpoint = {
        'epoch': epoch,
        'generator_state_dict': generator.state_dict(),
        'critic_state_dict': critic.state_dict(),
        'opt_g_state_dict': opt_g.state_dict(),
        'opt_c_state_dict': opt_c.state_dict(),
    }
    torch.save(checkpoint, path)

def load_checkpoint(generator, critic, opt_g, opt_c, path="/content/drive/MyDrive/gan_checkpoint.pth"):
    """Carrega o estado do treinamento de um checkpoint."""
    start_epoch = 1
    if os.path.exists(path):
        print(f"==> Carregando checkpoint de '{path}'...")
        checkpoint = torch.load(path)
        generator.load_state_dict(checkpoint['generator_state_dict'])
        critic.load_state_dict(checkpoint['critic_state_dict'])
        opt_g.load_state_dict(checkpoint['opt_g_state_dict'])
        opt_c.load_state_dict(checkpoint['opt_c_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"==> Checkpoint carregado. Reiniciando da época {start_epoch}")
    else:
        print("==> Nenhum checkpoint encontrado. Iniciando do zero.")
    return start_epoch

### WGAN:

#### Hiperparâmetros:

In [ ]:
def get_hyperparameters():
    """
    Retorna um dicionário de hiperparâmetros para o experimento WCGAN.
    """
    return {
        'z_size': 100,
        'num_epochs': 25, # 50 para o gerador do lab 6, 25 para essa tarefa
        'batch_size': 64,
        'lr_g' : 0.0002,
        'lr_d' : 0.0002,
        'n_critic': 5,
        'lambda_gp': 10,
        'n_classes': 10,
        'embedding_dim': 10
    }

#### Gerador:

In [ ]:
class Gerador(nn.Module):
    def __init__(self, z_dim, n_classes, embedding_dim):
        super().__init__()
        self.label_embedding = nn.Embedding(n_classes, embedding_dim)
        self.net = nn.Sequential(
            nn.Linear(z_dim + embedding_dim, 7*7*256), # 256 para o lab 6, 128 para esse lab
            nn.Unflatten(dim=1, unflattened_size=(256, 7, 7)),
            nn.BatchNorm2d(256),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.ReLU(True),
            nn.BatchNorm2d(128),
            nn.ConvTranspose2d(128, 1, kernel_size=5, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, z, labels):
        c = self.label_embedding(labels)
        x = torch.cat([z, c], dim=1)
        return self.net(x)

#### Crítico:

In [ ]:
class Critico(nn.Module):
    def __init__(self, n_classes, embedding_dim):
        super().__init__()
        self.label_embedding = nn.Embedding(n_classes, embedding_dim)
        self.conv_net = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, kernel_size=5, stride=2, padding=2),
            nn.LeakyReLU(0.2),
            nn.Flatten(),
        )
        self.final_layer = nn.Linear(7*7*128 + embedding_dim, 1)

    def forward(self, x, labels):
        conv_out = self.conv_net(x)
        c = self.label_embedding(labels)
        combined = torch.cat([conv_out, c], dim=1)
        return self.final_layer(combined)

#### Treinamento:

In [ ]:
def compute_gradient_penalty(critic, real_samples, fake_samples, labels, device):
    """Calculates the gradient penalty for a CONDITIONAL WGAN-GP."""
    alpha = torch.rand(real_samples.size(0), 1, 1, 1, device=device)
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    critic_interpolates = critic(interpolates, labels)

    gradients = torch.autograd.grad(
        outputs=critic_interpolates,
        inputs=interpolates,
        grad_outputs=torch.ones_like(critic_interpolates),
        create_graph=True,
        retain_graph=True,
    )[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

In [ ]:
def train_epoch(dataloader, generator, critic, opt_g, opt_c, params, epoch):
    """Executa uma única época de treinamento para a WCGAN."""
    generator.train()
    critic.train()

    losses_c, losses_g = [], []

    for real_data, labels in dataloader:
        batch_size = real_data.shape[0]
        real_data, labels = real_data.to(device), labels.to(device)

        # --- Treinamento do Crítico ---
        for _ in range(params['n_critic']):
            z = torch.randn(batch_size, params['z_size'], device=device)
            fake_data = generator(z, labels).detach()

            critic_real = critic(real_data, labels).mean()
            critic_fake = critic(fake_data, labels).mean()
            loss_c_original = critic_fake - critic_real

            penalty = compute_gradient_penalty(critic, real_data, fake_data, labels, device)

            loss_c = loss_c_original + params['lambda_gp'] * penalty

            opt_c.zero_grad()
            loss_c.backward()
            opt_c.step()

        losses_c.append(loss_c.item())

        # --- Treinamento do Gerador ---
        z = torch.randn(batch_size, params['z_size'], device=device)
        fake_data = generator(z, labels)
        critic_fake_for_g = critic(fake_data, labels)

        loss_g = -critic_fake_for_g.mean()

        opt_g.zero_grad()
        loss_g.backward()
        opt_g.step()

        losses_g.append(loss_g.item())

    avg_loss_c = np.mean(losses_c)
    avg_loss_g = np.mean(losses_g)
    print(f"[Época {epoch:02d}] Perda Média Crítico: {avg_loss_c:.4f} | Perda Média Gerador: {avg_loss_g:.4f}")

    return avg_loss_c, avg_loss_g

In [ ]:
def laco_de_treinamento(dataloader, generator, critic, params):
    """Executa o loop de treinamento completo e retorna o histórico de perdas."""
    history_c, history_g = [], []

    optimizer_g = optim.Adam(generator.parameters(), lr=params['lr_g'], betas=(0.5, 0.9))
    optimizer_c = optim.Adam(critic.parameters(), lr=params['lr_d'], betas=(0.5, 0.9))

    start_epoch = load_checkpoint(generator, critic, optimizer_g, optimizer_c)

    print("Iniciando o treinamento da WCGAN...")
    for epoch in range(start_epoch, params['num_epochs'] + 1):
        print(f"epoch: {epoch}")
        loss_c, loss_g = train_epoch(dataloader, generator, critic, optimizer_g, optimizer_c, params, epoch)
        history_c.append(loss_c)
        history_g.append(loss_g)

        save_checkpoint(epoch, generator, critic, optimizer_g, optimizer_c)

    print("\nTreinamento concluído.")
    return history_g, history_c

### Geração e Interpolação:

In [ ]:
@torch.no_grad()
def generate_images_for_each_class(generator, z_dim, n_classes=10):
    """Gera uma imagem para cada classe usando o mesmo vetor de ruído."""
    generator.eval()
    z = torch.randn(1, z_dim, device=device).repeat(n_classes, 1)
    labels = torch.arange(n_classes, device=device)
    fake_data = generator(z, labels).cpu().numpy()
    return [img for img in fake_data]

In [ ]:
@torch.no_grad()
def generate_with_fixed_noise(generator, z_dim, n_classes=10):
    """Gera imagens para todas as classes a partir de um único vetor de ruído fixo."""
    generator.eval()
    z_fixed = torch.randn(1, z_dim, device=device)
    labels = torch.arange(n_classes, device=device)
    return [generator(z_fixed, lbl.unsqueeze(0)).cpu().squeeze().numpy() for lbl in labels]

In [ ]:
@torch.no_grad()
def interpolate_noise(generator, z_dim, fixed_class, steps=10):
    """Mantém a classe fixa e interpola entre dois vetores de ruído."""
    generator.eval()
    z1, z2 = torch.randn(1, z_dim, device=device), torch.randn(1, z_dim, device=device)
    label = torch.tensor([fixed_class], device=device)
    images = []
    for alpha in torch.linspace(0, 1, steps):
        z_interp = (1 - alpha) * z1 + alpha * z2
        images.append(generator(z_interp, label).cpu().squeeze().numpy())

In [ ]:
@torch.no_grad()
def interpolate_classes(generator, z_dim, c1, c2, steps=10):
    """Mantém o ruído fixo e interpola entre os embeddings de duas classes."""
    generator.eval()
    z_fixed = torch.randn(1, z_dim, device=device)
    embed_c1 = generator.label_embedding(torch.tensor([c1], device=device))
    embed_c2 = generator.label_embedding(torch.tensor([c2], device=device))
    images = []
    for alpha in torch.linspace(0, 1, steps):
        embed_interp = (1 - alpha) * embed_c1 + alpha * embed_c2
        combined_input = torch.cat([z_fixed, embed_interp], dim=1)
        images.append(generator.net(combined_input).cpu().squeeze().numpy())
    return images

In [ ]:
@torch.no_grad()
def interpolate_both(generator, z_dim, c1, c2, steps=10):
    """Interpola simultaneamente entre dois vetores de ruído e duas classes."""
    generator.eval()
    z1, z2 = torch.randn(1, z_dim, device=device), torch.randn(1, z_dim, device=device)
    label1, label2 = torch.tensor([c1], device=device), torch.tensor([c2], device=device)
    embed_c1 = generator.label_embedding(label1)
    embed_c2 = generator.label_embedding(label2)
    images = []
    for alpha in torch.linspace(0, 1, steps):
        z_interp = (1 - alpha) * z1 + alpha * z2
        embed_interp = (1 - alpha) * embed_c1 + alpha * embed_c2
        combined_input = torch.cat([z_interp, embed_interp], dim=1)
        images.append(generator.net(combined_input).cpu().squeeze().numpy())
    return images

### Salvando e carregando:

In [ ]:
def save_model(model, name, dataset_choice):
    """Salva o modelo do gerador treinado num arquivo."""
    model_path = f'files/{dataset_choice}_{name}_conv_wgan.pt'
    torch.jit.script(model).save(model_path)
    print(f"\nModelo final salvo em: {model_path}")
    return model_path

In [ ]:
def load_model(model_path, device):
    """Carrega um modelo salvo e o configura para o modo de avaliação."""
    loaded_model = torch.jit.load(model_path, map_location=device)
    loaded_model.eval()
    return loaded_model

### Executando:

In [ ]:
# --- Hiperparâmetros ---
params = get_hyperparameters()

# --- Preparar Dados e Modelos ---
train_loader = prepare_data(params['batch_size'], 'FMNIST')
modelo_gerador = Gerador(params['z_size'], params['n_classes'], params['embedding_dim']).to(device)
modelo_critico = Critico(params['n_classes'], params['embedding_dim']).to(device)

# --- Treinamento ---
loss_g_hist, loss_c_hist = laco_de_treinamento(train_loader, modelo_gerador, modelo_critico, params)
save_model(modelo_gerador, 'conditional', 'fmnist')

1a:

In [ ]:
## Tarefa 1a: Gráficos das funções de perda
print("\n--- Tarefa 1a: Gráfico de Perdas ---")
plot_loss_history(loss_g_hist, loss_c_hist)

1b:

In [ ]:
## Tarefa 1b: Imagens geradas com o modelo
print("\n--- Tarefa 1b: Imagens Geradas (uma para cada classe) ---")
imagens_por_classe = generate_images_for_each_class(modelo_gerador, params['z_size'])
titulos = [FMNIST_LABELS[i] for i in range(len(imagens_por_classe))]
plot_image_grid(imagens_por_classe, titles=titulos, n_cols=5, figsize=(12, 6))

2a:

In [ ]:
## Tarefa 2a: Interpolação entre classes (z fixo)
print("\n--- Tarefa 2a: Interpolação entre Classes ---")
c1, c2 = 5, 9  # Sandal -> Ankle Boot
imagens_interp_c = interpolate_classes(modelo_gerador, params['z_size'], c1, c2, steps=10)
print(f"Interpolação: {FMNIST_LABELS[c1]} -> {FMNIST_LABELS[c2]}")
plot_image_grid(imagens_interp_c, n_cols=10, figsize=(10, 2))

2b:

In [ ]:
## Tarefa 2b: Interpolação entre vetores de ruído (c fixo)
print("\n--- Tarefa 2b: Interpolação de Ruído ---")
c_fixa = 8 # Bag
imagens_interp_z = interpolate_noise(modelo_gerador, params['z_size'], c_fixa, steps=10)
print(f"Interpolação de ruído para a classe: {FMNIST_LABELS[c_fixa]}")
plot_image_grid(imagens_interp_z, n_cols=10, figsize=(10, 2))

2c:

In [ ]:
## Tarefa 2c: Interpolação de ruído e classes
print("\n--- Tarefa 2c: Interpolação de Ruído e Classes ---")
c1, c2 = 1, 7 # Trouser -> Sneaker
imagens_interp_total = interpolate_both(modelo_gerador, params['z_size'], c1, c2, steps=10)
print(f"Interpolação: {FMNIST_LABELS[c1]} -> {FMNIST_LABELS[c2]}")
plot_image_grid(imagens_interp_total, n_cols=10, figsize=(10, 2))

3:

In [ ]:
## Tarefa 3: Vetor de ruído fixo e alteração do rótulo
print("\n--- Tarefa 3: Ruído Fixo, Classes Variadas ---")
imagens_ruido_fixo = generate_with_fixed_noise(modelo_gerador, params['z_size'])
titulos_ruido_fixo = [FMNIST_LABELS[i] for i in range(len(imagens_ruido_fixo))]
plot_image_grid(imagens_ruido_fixo, titles=titulos_ruido_fixo, n_cols=10, figsize=(15, 3))

### Para o lab 6:

In [ ]:
# --- Hiperparâmetros ---
params = get_hyperparameters()

# --- Preparar Dados e Modelos ---
train_loader = prepare_data(params['batch_size'], 'MNIST')
modelo_gerador = Gerador(params['z_size'], params['n_classes'], params['embedding_dim']).to(device)
modelo_critico = Critico(params['n_classes'], params['embedding_dim']).to(device)

# --- Treinamento ---
loss_g_hist, loss_c_hist = laco_de_treinamento(train_loader, modelo_gerador, modelo_critico, params)
save_model(modelo_gerador, 'conditional', 'mnist')

In [ ]:
plot_loss_history(loss_g_hist, loss_c_hist)

In [ ]:
imagens_por_classe = generate_images_for_each_class(modelo_gerador, params['z_size'])
titulos = [MNIST_LABELS[i] for i in range(len(imagens_por_classe))]
plot_image_grid(imagens_por_classe, titles=titulos, n_cols=5, figsize=(12, 6))

# print

In [ ]:
!pip uninstall -y torchsummary
!pip install torch-summary

In [ ]:
from torchsummary import summary
params = get_hyperparameters()

modelo_gerador = Gerador(params['z_size'], params['n_classes'], params['embedding_dim']).to(device)
modelo_critico = Critico(params['n_classes'], params['embedding_dim']).to(device)

print("--- Generator Summary ---")
summary(
    modelo_gerador,
    input_size=[(params['z_size'],), (1,)],
    dtypes=[torch.FloatTensor, torch.LongTensor],
    device=device,
    verbose=2
)

print("\n" + "="*60 + "\n")

print("--- Critic Summary ---")
summary(
    modelo_critico,
    input_size=[(1, 28, 28), (1,)],
    dtypes=[torch.FloatTensor, torch.LongTensor],
    device=device,
    verbose=2
)